In [29]:
import pandas as pd
import math

In [30]:
df = pd.read_csv("ActLab_df.csv")

In [31]:
# Initialize the dictionary with each key as "INPUT|CONTROLLER|x_a" where x_a is activation type, and value being a list of 
# [num_mentions, pos_or_neg]

interaction_dict = {}
i = 0
for i in range(0, len(df.INPUT)):
    if df.NUM_LABEL[i] == 1:
        interaction_dict[f"{df.INPUT[i]}|{df.CONTROLLER[i]}|pos"] = [df.COUNTER[i], df.NUM_LABEL[i]]
    elif df.NUM_LABEL[i] == -1:
        interaction_dict[f"{df.OUTPUT[i]}|{df.CONTROLLER[i]}|neg"] = [df.COUNTER[i], df.NUM_LABEL[i]]
    i += 1

In [32]:
# We make a proportion dictionary with each key as "Input/Output|Controller" and each value as [pos_count, total_count]

# initialize the dictionary
prop_dict = {}

for key in interaction_dict:
    prop_dict[key[:-4]] = [0, 0]

In [33]:
# Add just Input/Output|Controller as key to prop_dict, and if the activation is postive, add to the first element in the list value. Otherwise
# just add to total.

for key in interaction_dict:
    split_key = key.split("|")
    count = interaction_dict[key][0]
    if split_key[2] == "pos":
        prop_dict[key[:-4]][0] += count
        prop_dict[key[:-4]][1] += count
    if split_key[2] == "neg":
        prop_dict[key[:-4]][1] += count

In [34]:
# for each entry in prop_dict, divide pos by total to get the proportion. Then input into the modified tanh function to get our edge value.
# subtract 1 for tanh, leave -1 out for sigmoid

for key in prop_dict:
    proportion = prop_dict[key][0] / prop_dict[key][1]
    y = (1/(1+math.exp(-5*(proportion-.5))))
    prop_dict[key].append(y)

In [35]:
# Change dictionary's form into dataframe-compatible form.
df_dict = {
    "INPUT": [],
    "CONTROLLER": [],
    "EDGE": [],
    "POS": [],
    "TOTAL": [],
}
for key in prop_dict:
    met_input, met_cont = key.split("|")
    df_dict["INPUT"].append(met_input)
    df_dict["CONTROLLER"].append(met_cont)
    df_dict["EDGE"].append(prop_dict[key][2])
    df_dict["POS"].append(prop_dict[key][0])
    df_dict["TOTAL"].append(prop_dict[key][1])


In [36]:
prop_df = pd.DataFrame.from_dict(df_dict)

In [37]:
i = 0
input_spec = prop_df["INPUT"]
cont_spec = prop_df["CONTROLLER"]
ID_PAIRS = {"ID_PAIRS": []}

for i in range(0, len(input_spec)):
    if len(input_spec[i]) > 5:
        input_spec_id = input_spec[i].split(":")[-2] + ":" + input_spec[i].split(":")[-1]
    if len(cont_spec[i]) > 5:
        cont_spec_id = cont_spec[i].split(":")[-2] + ":" + cont_spec[i].split(":")[-1]
    else:
        input_spec_id = input_spec
        cont_spec_id = cont_spec

    ID_PAIRS["ID_PAIRS"].append(f"{input_spec_id}|{cont_spec_id}")
    i += 1

In [38]:
ID_PAIRS_df = pd.DataFrame.from_dict(ID_PAIRS)
prop_df = pd.concat([prop_df, ID_PAIRS_df], axis=1, join="inner")
with_id = pd.concat([prop_df["INPUT"], prop_df["CONTROLLER"], ID_PAIRS_df], axis=1, join="inner")
with_id = with_id.drop_duplicates(subset="ID_PAIRS", inplace=False)

In [39]:
unique_dict = {}
for pair in ID_PAIRS_df["ID_PAIRS"]:
    unique_dict[pair] = [0, 0]

i = 0
for i in range(len(prop_df["ID_PAIRS"])):
    id_pair = prop_df["ID_PAIRS"][i]
    unique_dict[id_pair][0] += prop_df["POS"][i]
    unique_dict[id_pair][1] += prop_df["TOTAL"][i]

In [40]:
pos_t_dict = {"POS" : [], "TOTAL": [], "EDGE": []}

for key in unique_dict:
    pos_t_dict["POS"].append(unique_dict[key][0])
    pos_t_dict["TOTAL"].append(unique_dict[key][1])

    proportion = unique_dict[key][0] / unique_dict[key][1]
    y = (1/(1+math.exp(-5*(proportion-.5))))
    
    pos_t_dict["EDGE"].append(y)

In [41]:
pos_t_df = pd.DataFrame.from_dict(pos_t_dict)
with_id = with_id.reset_index(drop=True)
prop_df_reduced = pd.concat([with_id, pos_t_df], axis=1, join="inner")

In [42]:
prop_df_reduced

,INPUT,CONTROLLER,ID_PAIRS,POS,TOTAL,EDGE
0,apoptosis::go:GO:0006915,ROS::chebi:CHEBI:26523,GO:0006915|CHEBI:26523,82,86,0.906144
1,inflammation::go:GO:0006954,LPS::uniprot:O14896,GO:0006954|uniprot:O14896,70,82,0.854245
2,insulin secretion::go:GO:0030073,glucose::chebi:CHEBI:17234,GO:0030073|CHEBI:17234,44,53,0.839019
3,FBS::uniprot:Q9HAH7,heat::pfam:PF02985,uniprot:Q9HAH7|pfam:PF02985,0,38,0.075858
4,cell death::go:GO:0008219,ROS::chebi:CHEBI:26523,GO:0008219|CHEBI:26523,37,39,0.904096
...,...,...,...,...,...,...
93836,‐8::uaz:UAZE2809038,mycoplasma infection::mesh:D009175,uaz:UAZE2809038|mesh:D009175,1,1,0.924142
93837,′-CTGCTGGATGACGTGAGGT AACC-3′::uaz:UAZE280B22D...,probe (5′-VIC TGTCTCGCTCCGTGGCCTTAGCTG-3′::uaz...,uaz:UAZE280B22D4354474354474741544741434754474...,0,1,0.075858
93838,’s::uaz:UAZE2809973,MVs::pubchem:449,uaz:UAZE2809973|pubchem:449,1,1,0.924142
93839,♂::uaz:UAZE29982,cecropin::pfam:PF00272,uaz:UAZE29982|pfam:PF00272,1,1,0.924142


In [49]:
in_cont_id = {"INPUT_ID" : [], "CONT_ID" : []}
for pair in prop_df_reduced["ID_PAIRS"]:
    in_id, cont_id = pair.split("|") 
    in_cont_id["INPUT_ID"].append(in_id)
    in_cont_id["CONT_ID"].append(cont_id)
in_cont_id_df = pd.DataFrame.from_dict(in_cont_id)
with_ids = pd.concat([in_cont_id_df, prop_df_reduced], axis=1)

In [50]:
with_ids

,INPUT_ID,CONT_ID,INPUT,CONTROLLER,ID_PAIRS,POS,TOTAL,EDGE
0,GO:0006915,CHEBI:26523,apoptosis::go:GO:0006915,ROS::chebi:CHEBI:26523,GO:0006915|CHEBI:26523,82,86,0.906144
1,GO:0006954,uniprot:O14896,inflammation::go:GO:0006954,LPS::uniprot:O14896,GO:0006954|uniprot:O14896,70,82,0.854245
2,GO:0030073,CHEBI:17234,insulin secretion::go:GO:0030073,glucose::chebi:CHEBI:17234,GO:0030073|CHEBI:17234,44,53,0.839019
3,uniprot:Q9HAH7,pfam:PF02985,FBS::uniprot:Q9HAH7,heat::pfam:PF02985,uniprot:Q9HAH7|pfam:PF02985,0,38,0.075858
4,GO:0008219,CHEBI:26523,cell death::go:GO:0008219,ROS::chebi:CHEBI:26523,GO:0008219|CHEBI:26523,37,39,0.904096
...,...,...,...,...,...,...,...,...
93836,uaz:UAZE2809038,mesh:D009175,‐8::uaz:UAZE2809038,mycoplasma infection::mesh:D009175,uaz:UAZE2809038|mesh:D009175,1,1,0.924142
93837,uaz:UAZE280B22D4354474354474741544741434754474...,uaz:UAZ70726F6265202835E280B22D564943205447544...,′-CTGCTGGATGACGTGAGGT AACC-3′::uaz:UAZE280B22D...,probe (5′-VIC TGTCTCGCTCCGTGGCCTTAGCTG-3′::uaz...,uaz:UAZE280B22D4354474354474741544741434754474...,0,1,0.075858
93838,uaz:UAZE2809973,pubchem:449,’s::uaz:UAZE2809973,MVs::pubchem:449,uaz:UAZE2809973|pubchem:449,1,1,0.924142
93839,uaz:UAZE29982,pfam:PF00272,♂::uaz:UAZE29982,cecropin::pfam:PF00272,uaz:UAZE29982|pfam:PF00272,1,1,0.924142


In [51]:
with_ids.to_csv("Act_tanh.csv")